In [12]:
train_path = 'train_hand_fin.txt'
dev_path = 'dev_hand_fin.txt'
test_path = 'test_hand_fin.txt'
train_path_2 = 'skweak_all.txt'
model_checkpoint = "bert-base-german-cased"

batch_size = 16
learning_rate = 5e-5
epoch = 4
weight_decay = 0.1

save_folder_name = 'bert_alldata'
save_model_name = 'bert_alldata_fin'

In [2]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 20.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
import re
import pandas as pd
from datasets import Dataset

Load dataset

In [4]:
def load_data(path):
  file = open(path, 'r')
  lines = file.readlines()

  tokens = []
  ner_tags = []

  temp_toks = []
  temp_tags = []

  for i in lines:
    if i == '\n':
      tokens.append(temp_toks)
      ner_tags.append(temp_tags)

      temp_toks = []
      temp_tags = []

    else:
      temp = re.split('\t',i)
      temp_toks.append(temp[0])
      temp_tags.append(re.sub('\n','',temp[-1]))
  
  data = pd.DataFrame({'tokens':tokens,
                       'ner_tags':ner_tags})
  return data


In [5]:
train1 = load_data(train_path)
train2 = load_data(train_path_2)
traindata = pd.concat([train1,train2])
traindata = traindata.reset_index(drop = True)
test = load_data(test_path)
dev = load_data(dev_path)

traindata.head()

,tokens,ner_tags
0,[ABKÜRZUNGSVERZEICHNIS],[O]
1,"[Empfehlung, Nr., 10, Ein, Überblick, der, Akt...","[O, O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, O..."
2,"[Betreutes, Konto, Ist, ein, von, der, Schuldn...","[O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, I-ORG,..."
3,"[Insgesamt, verfügte, die, Rettungsleitstelle,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[Empfehlungen, ,, die, in, einem, früheren, Be...","[O, O, O, O, O, O, O, O, B-ORG, I-ORG, O, O, O..."


In [6]:
label2id = {'O': 0,'B-ORG':1,'I-ORG':2}
id2label = {0 :'O', 1:'B-ORG', 2:'I-ORG'}
label_list = ['O','B-ORG','I-ORG']

In [13]:
from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer
)

In [14]:
config = AutoConfig.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label = id2label,
    label2id = label2id
)


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    config=config
)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/485k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-b

In [16]:
def align_labels(data, label_encoding_dict):
    tokenized_inputs = tokenizer(data["tokens"],
                        max_length = 128, padding = 'max_length',
                        truncation=True, is_split_into_words=True)

    label_id_temp = {}
    for i, label in enumerate(data['ner_tags']):
        label_id_temp.update({i:label})

    word_ids = tokenized_inputs.word_ids(batch_index=0)

    labels = []
    for w_id in word_ids:
        if w_id == None:
            labels.append(-100)
        else:
            tag = label_id_temp[w_id]
            labels.append(label2id[tag])

    tokenized_inputs["labels"] = labels

    return tokenized_inputs

def tokenized_for_bert(df):
  for_bert = df.copy()
  for_bert['input_ids'] = ""
  for_bert['token_type_ids'] = ""
  for_bert['attention_mask']= ""
  for_bert['labels'] = ""

  for index, row in df.iterrows():
      inputs = align_labels(row,label2id)
      for_bert.at[index,'input_ids'] = inputs['input_ids']
      for_bert.at[index,'token_type_ids'] = inputs['token_type_ids']
      for_bert.at[index,'attention_mask'] = inputs['attention_mask']
      for_bert.at[index,'labels'] = inputs['labels']
  for_bert = for_bert.drop('tokens', axis = 1)
  for_bert = for_bert.drop('ner_tags', axis = 1)
  dataset_for_bert = Dataset.from_pandas(for_bert)
  print(dataset_for_bert)
  return dataset_for_bert

In [ ]:
train_dataset = tokenized_for_bert(traindata)
dev_dataset = tokenized_for_bert(dev)
test_dataset = tokenized_for_bert(test)

In [19]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=5ab87bbd35d3b765bddcaa355b12b555f11ff9828bac83ae4684bdaeec3a7559
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [20]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch
from seqeval.metrics import classification_report

In [21]:
import numpy as np
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    
    result = classification_report(true_labels, true_predictions, output_dict = True)

    print(result)
    
    return result['ORG']

In [22]:
args = TrainingArguments(
    save_folder_name,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay = weight_decay,
    metric_for_best_model = 'f1-score',
    save_total_limit=5,
    load_best_model_at_end = True
)

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("Training...")
trainer.train()
print("Evaluating...")
trainer.evaluate()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3678
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 920
  Number of trainable parameters = 108493059
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Training...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1-score,Support
1,0.074400,0.050473,0.706526,0.830334,0.763443,2334
2,0.037100,0.044692,0.783877,0.874893,0.826888,2334
3,0.023500,0.046409,0.819048,0.847472,0.833017,2334
4,0.011900,0.055448,0.797434,0.878749,0.836119,2334


***** Running Evaluation *****
  Num examples = 1227
  Batch size = 16


{'ORG': {'precision': 0.7065257017863653, 'recall': 0.8303341902313625, 'f1-score': 0.7634429781366948, 'support': 2334}, 'micro avg': {'precision': 0.7065257017863653, 'recall': 0.8303341902313625, 'f1-score': 0.7634429781366948, 'support': 2334}, 'macro avg': {'precision': 0.7065257017863653, 'recall': 0.8303341902313625, 'f1-score': 0.7634429781366948, 'support': 2334}, 'weighted avg': {'precision': 0.7065257017863653, 'recall': 0.8303341902313625, 'f1-score': 0.7634429781366948, 'support': 2334}}


...

{'ORG': {'precision': 0.7974339035769828, 'recall': 0.8787489288774636, 'f1-score': 0.8361190379127598, 'support': 2334}, 'micro avg': {'precision': 0.7974339035769828, 'recall': 0.8787489288774636, 'f1-score': 0.8361190379127598, 'support': 2334}, 'macro avg': {'precision': 0.7974339035769828, 'recall': 0.8787489288774636, 'f1-score': 0.8361190379127598, 'support': 2334}, 'weighted avg': {'precision': 0.7974339035769828, 'recall': 0.8787489288774636, 'f1-score': 0.8361190379127598, 'support': 2334}}


Saving model checkpoint to bert_alldata/checkpoint-920
Configuration saved in bert_alldata/checkpoint-920/config.json
Model weights saved in bert_alldata/checkpoint-920/pytorch_model.bin
tokenizer config file saved in bert_alldata/checkpoint-920/tokenizer_config.json
Special tokens file saved in bert_alldata/checkpoint-920/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from bert_alldata/checkpoint-920 (score: 0.8361190379127598).
***** Running Evaluation *****
  Num examples = 1227
  Batch size = 16


Evaluating...


{'ORG': {'precision': 0.7974339035769828, 'recall': 0.8787489288774636, 'f1-score': 0.8361190379127598, 'support': 2334}, 'micro avg': {'precision': 0.7974339035769828, 'recall': 0.8787489288774636, 'f1-score': 0.8361190379127598, 'support': 2334}, 'macro avg': {'precision': 0.7974339035769828, 'recall': 0.8787489288774636, 'f1-score': 0.8361190379127598, 'support': 2334}, 'weighted avg': {'precision': 0.7974339035769828, 'recall': 0.8787489288774636, 'f1-score': 0.8361190379127598, 'support': 2334}}


{'eval_loss': 0.05544842407107353,
 'eval_precision': 0.7974339035769828,
 'eval_recall': 0.8787489288774636,
 'eval_f1-score': 0.8361190379127598,
 'eval_support': 2334,
 'eval_runtime': 10.313,
 'eval_samples_per_second': 118.976,
 'eval_steps_per_second': 7.466,
 'epoch': 4.0}

In [23]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 1226
  Batch size = 16


{'ORG': {'precision': 0.8174982911825017, 'recall': 0.9012810851544838, 'f1-score': 0.8573476702508961, 'support': 2654}, 'micro avg': {'precision': 0.8174982911825017, 'recall': 0.9012810851544838, 'f1-score': 0.8573476702508961, 'support': 2654}, 'macro avg': {'precision': 0.8174982911825017, 'recall': 0.9012810851544838, 'f1-score': 0.8573476702508961, 'support': 2654}, 'weighted avg': {'precision': 0.8174982911825017, 'recall': 0.9012810851544838, 'f1-score': 0.8573476702508962, 'support': 2654}}


PredictionOutput(predictions=array([[[ 7.2887316, -2.9895403, -3.9738452],
        [ 7.494145 , -2.5245187, -3.736979 ],
        [ 7.258362 , -2.545912 , -4.115741 ],
        ...,
        [ 8.006289 , -3.2128363, -4.263497 ],
        [ 7.9631996, -3.413719 , -3.893711 ],
        [ 7.749478 , -3.0941675, -3.9063134]],

       [[ 7.1517234, -3.2009714, -3.7157886],
        [ 6.337584 , -2.7375283, -3.7427583],
        [-2.1414847,  4.6593633, -2.3992357],
        ...,
        [ 3.8412228, -1.815338 , -1.3899215],
        [ 7.522485 , -3.308264 , -3.6791859],
        [ 6.279533 , -3.0170953, -3.005627 ]],

       [[ 7.710251 , -2.990319 , -3.919775 ],
        [ 8.555824 , -4.0040536, -4.0647426],
        [ 8.314909 , -3.0896425, -4.4721394],
        ...,
        [ 8.078798 , -2.9520977, -4.803867 ],
        [ 7.930398 , -2.827592 , -4.100646 ],
        [ 4.603826 , -1.1132784, -2.8742356]],

       ...,

       [[ 7.425429 , -3.1628132, -4.2238774],
        [ 8.568881 , -4.190814 , -4.220